In [4]:
import pandas as pd
import numpy as np
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense, Dropout, BatchNormalization

# Classification Problem

In [5]:
import pandas as pd
dataset = pd.read_csv("/content/drive/MyDrive/Adv. Ai by Sundaram- 9th Sep 2023-20240207T111312Z-001/Adv. Ai by Sundaram- 9th Sep 2023/16th and 17th Sep 2023/Churn_Modelling.csv")
dataset.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [6]:
dataset['Exited'].value_counts()

Exited
0    7963
1    2037
Name: count, dtype: int64

In [7]:
dataset = dataset.iloc[:,3:]
dataset.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [8]:
dataset.isnull().sum()

CreditScore        0
Geography          0
Gender             0
Age                0
Tenure             0
Balance            0
NumOfProducts      0
HasCrCard          0
IsActiveMember     0
EstimatedSalary    0
Exited             0
dtype: int64

In [9]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   CreditScore      10000 non-null  int64  
 1   Geography        10000 non-null  object 
 2   Gender           10000 non-null  object 
 3   Age              10000 non-null  int64  
 4   Tenure           10000 non-null  int64  
 5   Balance          10000 non-null  float64
 6   NumOfProducts    10000 non-null  int64  
 7   HasCrCard        10000 non-null  int64  
 8   IsActiveMember   10000 non-null  int64  
 9   EstimatedSalary  10000 non-null  float64
 10  Exited           10000 non-null  int64  
dtypes: float64(2), int64(7), object(2)
memory usage: 859.5+ KB


In [10]:
dataset = pd.get_dummies(dataset, columns=['Geography', 'Gender'], drop_first=True)

In [11]:
dataset.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_Germany,Geography_Spain,Gender_Male
0,619,42,2,0.00,1,1,1,101348.88,1,False,False,False
1,608,41,1,83807.86,1,0,1,112542.58,0,False,True,False
2,502,42,8,159660.80,3,1,0,113931.57,1,False,False,False
3,699,39,1,0.00,2,0,0,93826.63,0,False,False,False
4,850,43,2,125510.82,1,1,1,79084.10,0,False,True,False


In [12]:
# split the data into ind and dep variable
x = dataset.drop(['Exited'], axis=1)
y = dataset[['Exited']]

In [13]:
x.shape

(10000, 11)

In [14]:
x.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_Germany,Geography_Spain,Gender_Male
0,619,42,2,0.00,1,1,1,101348.88,False,False,False
1,608,41,1,83807.86,1,0,1,112542.58,False,True,False
2,502,42,8,159660.80,3,1,0,113931.57,False,False,False
3,699,39,1,0.00,2,0,0,93826.63,False,False,False
4,850,43,2,125510.82,1,1,1,79084.10,False,True,False


In [15]:
y.head()

,Exited
0,1
1,0
2,1
3,0
4,0


In [16]:
# Balance the data
from imblearn.over_sampling import SMOTE
smote = SMOTE()
x_smote, y_smote = smote.fit_resample(x, y)
print(y.value_counts())
print(y_smote.value_counts())


Exited
0         7963
1         2037
Name: count, dtype: int64
Exited
0         7963
1         7963
Name: count, dtype: int64


In [17]:
# split the data into train and test
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x_smote, y_smote, test_size=0.25, random_state=101)

In [18]:
# feature scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)

# MLP

In [19]:
model = Sequential()
model.add(Dense(32, activation='relu', kernel_regularizer = keras.regularizers.l2(0.03), kernel_initializer='he_normal',  input_dim=11))
model.add(Dropout(0.5))
model.add(BatchNormalization())

model.add(Dense(32, activation='relu',  kernel_regularizer = keras.regularizers.l2(0.03), kernel_initializer='he_normal'))
model.add(Dropout(0.5))
model.add(BatchNormalization())

model.add(Dense(1, activation='sigmoid'))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 32)                384       
                                                                 
 dropout (Dropout)           (None, 32)                0         
                                                                 
 batch_normalization (Batch  (None, 32)                128       
 Normalization)                                                  
                                                                 
 dense_1 (Dense)             (None, 32)                1056      
                                                                 
 dropout_1 (Dropout)         (None, 32)                0         
                                                                 
 batch_normalization_1 (Bat  (None, 32)                128       
 chNormalization)                                       

In [20]:
initial_weights = model.get_weights()

In [21]:
initial_weights[0]

array([[-0.15081127,  0.05302669,  0.09500646,  0.27553666,  0.13301629,
        -0.13426971, -0.05768599,  0.80392087, -0.45170674, -0.20078589,
         0.5330381 ,  0.11722702, -0.01390054,  0.13157494,  0.23258652,
        -0.68946695, -0.44659415,  0.3320397 ,  0.76617396,  0.00766661,
         0.56529623, -0.30073595,  0.22120912,  0.05968828, -0.37384346,
         0.62876   ,  0.6182266 , -0.36618173, -0.37044233, -0.02575282,
         0.24779446, -0.39357278],
       [ 0.5264167 ,  0.8582376 , -0.3043804 , -0.66714746,  0.06371284,
         0.00199385,  0.11848576, -0.46996635,  0.3656752 , -0.5047334 ,
         0.3405499 ,  0.8946628 , -0.33078915,  0.5463149 ,  0.77529234,
        -0.4982101 , -0.00810972, -0.18103917,  0.29515988, -0.13581018,
        -0.45646214, -0.5358614 , -0.2570175 ,  0.25418514, -0.06104314,
        -0.24764806, -0.86248314, -0.730297  ,  0.91707397, -0.7562501 ,
         0.57852006, -0.18697496],
       [ 0.60681844,  0.07174365,  0.42221534, -0.3424

In [22]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'] )

In [23]:
history = model.fit(x_train, y_train, batch_size=32, epochs=100, validation_data=(x_test, y_test))

Epoch 1/100
374/374 [==============================] - 9s 6ms/step - loss: 2.7519 - accuracy: 0.6260 - val_loss: 1.4761 - val_accuracy: 0.7657
Epoch 2/100
374/374 [==============================] - 2s 5ms/step - loss: 1.0752 - accuracy: 0.7189 - val_loss: 0.7335 - val_accuracy: 0.7808
Epoch 3/100
374/374 [==============================] - 2s 5ms/step - loss: 0.6542 - accuracy: 0.7585 - val_loss: 0.5296 - val_accuracy: 0.7958
Epoch 4/100
374/374 [==============================] - 2s 5ms/step - loss: 0.5334 - accuracy: 0.7785 - val_loss: 0.4591 - val_accuracy: 0.8066
Epoch 5/100
374/374 [==============================] - 3s 7ms/step - loss: 0.5035 - accuracy: 0.7878 - val_loss: 0.4381 - val_accuracy: 0.8172
Epoch 6/100
374/374 [==============================] - 3s 7ms/step - loss: 0.4854 - accuracy: 0.7965 - val_loss: 0.4281 - val_accuracy: 0.8204
Epoch 7/100
374/374 [==============================] - 2s 5ms/step - loss: 0.4906 - accuracy: 0.7941 - val_loss: 0.4218 - val_accuracy: 0.8197